In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker

In [2]:
# --- Funções auxiliares ---

def formatar_dataframe(df, titulo, coluna_esquerda=None, coluna_centro=None):
    df_styler = df.style.set_caption(titulo)
    if coluna_esquerda:
        df_styler.set_properties(subset=coluna_esquerda, **{'text-align': 'left'})
    if coluna_centro:
        df_styler.set_properties(subset=coluna_centro, **{'text-align': 'center'})
    df_styler.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
    return df_styler

def criar_tabela_frequencia(df, coluna, titulo, mapeamento=None, formato=None):
    counts = df[coluna].value_counts()
    if mapeamento:
        descricao = counts.index.map(mapeamento)
    else:
        descricao = counts.index
    tabela = pd.DataFrame({
        "Descrição": descricao,
        "Quantidade": counts.values,
        "Percentual": counts / len(df)
    })
    if formato:
        return formatar_dataframe(tabela, titulo, ["Descrição"], ["Quantidade", "Percentual"]).format(formato)
    else:
        return formatar_dataframe(tabela, titulo, ["Descrição"], ["Quantidade", "Percentual"]).format({'Percentual': "{:,.2%}"})

def criar_histograma(df, coluna, titulo, intervalos=None, cor_face=None, cor_edge=None):
    plt.figure(figsize=(8, 5))
    plt.hist(df[coluna], weights=np.ones_like(df[coluna]) / len(df), bins=90, facecolor=cor_face, edgecolor=cor_edge, linewidth=0.5)
    if intervalos:
        plt.xticks(intervalos, intervalos, rotation='vertical')
    plt.gca().yaxis.set_major_formatter(ticker.PercentFormatter(1, decimals=0))
    plt.title(titulo)
    plt.xlabel('Valores')
    plt.ylabel('Total (porcentagem)')
    plt.savefig(titulo.replace(" ", "_").replace(":", "") + ".png")
    plt.show()

def criar_grafico_barras(df, titulo, x, y, labels=None):
    plt.figure(figsize=(10, 5))
    ax = sns.barplot(data=df, x=x, y=y, alpha=1, width=0.70, dodge=False)
    plt.title(titulo)
    plt.gca().xaxis.set_major_formatter(ticker.PercentFormatter(1, decimals=0))
    if labels is not None:
        for c in ax.containers:
            ax.bar_label(c, labels=labels, padding=2, fontsize=7)
    sns.despine()
    plt.savefig(titulo.replace(" ", "_").replace(":", "") + ".png")
    plt.show()

In [3]:
# --- Carregar os dados ---

df_microdados_enem = pd.read_parquet("microdados_enem_2016.parquet", engine="fastparquet")

In [4]:
# --- Dicionários de mapeamento ---

situacao_conclusao_dict = {
    1: "Já concluí o Ensino Médio",
    2: "Estou cursando e concluirei o Ensino Médio em 2016",
    3: "Estou cursando e concluirei o Ensino Médio após 2016",
    4: "Não concluí e não estou cursando o Ensino Médio"
}

ano_conclusao_dict = {
    0: "Não informado", 1: "2015", 2: "2014", 3: "2013", 4: "2012",
    5: "2011", 6: "2010", 7: "2009", 8: "2008", 9: "2007", 10: "Anterior a 2007"
}

ensino_dict = {
    1: "Ensino Regular", 2: "Educação Especial - Modalidade Substitutiva", 3: "Educação de Jovens e Adultos"
}

tp_escola_dict = {
    1: "Não respondeu", 2: "Pública", 3: "Privada", 4: "Exterior"
}

tp_cor_raca_dict = {
    0: "Não declarado", 1: "Branca", 2: "Preta", 3: "Parda", 4: "Amarela", 5: "Indígena", 6: "Sem informação"
}


# --- PLACEHOLDERS no q046_map ---

q046_map = {
    "A": "Placeholder_A",
    "B": "Placeholder_B",
    "C": "Placeholder_C",
    "D": "Placeholder_D",
    "E": "Placeholder_E",
    "F": "Placeholder_F",
    "G": "Placeholder_G",
    "H": "Placeholder_H"
}

In [5]:
# --- Tabelas de frequência ---

display(criar_tabela_frequencia(df_microdados_enem, "TP_ST_CONCLUSAO", "Tabela 1: Tabulação da variável TP_ST_CONCLUSAO", situacao_conclusao_dict))
display(criar_tabela_frequencia(df_microdados_enem, "Q046", "Tabela 2: Tabulação da variável Q046", q046_map))
display(criar_tabela_frequencia(df_microdados_enem, "TP_ANO_CONCLUIU", "Tabela 3: Tabulação da variável TP_ANO_CONCLUIU", ano_conclusao_dict))
display(criar_tabela_frequencia(df_microdados_enem, "TP_ENSINO", "Tabela 4: Tabulação da variável TP_ENSINO", ensino_dict))

,Descrição,Quantidade,Percentual
TP_ST_CONCLUSAO,,,
1,Já concluí o Ensino Médio,4928251,57.12%
2,Estou cursando e concluirei o Ensino Médio em 2016,1882278,21.82%
3,Estou cursando e concluirei o Ensino Médio após 2016,1344085,15.58%
4,Não concluí e não estou cursando o Ensino Médio,472753,5.48%


,Descrição,Quantidade,Percentual
Q046,,,
A,Placeholder_A,4947935,57.35%
B,Placeholder_B,1872570,21.70%
C,Placeholder_C,1331073,15.43%
D,Placeholder_D,475785,5.51%


,Descrição,Quantidade,Percentual
TP_ANO_CONCLUIU,,,
0,Não informado,3699116,42.88%
10,Anterior a 2007,1083909,12.56%
1,2015,966842,11.21%
2,2014,699987,8.11%
3,2013,527310,6.11%
4,2012,416454,4.83%
5,2011,317364,3.68%
6,2010,294214,3.41%
7,2009,244461,2.83%


,Descrição,Quantidade,Percentual
TP_ENSINO,,,
1,Ensino Regular,1652485,19.15%
3,Educação de Jovens e Adultos,218532,2.53%
2,Educação Especial - Modalidade Substitutiva,10295,0.12%


In [6]:
# --- Tabela 5: Tamanho de CO_ESCOLA ---

tamanho_CO_ESCOLA = df_microdados_enem['CO_ESCOLA'].astype(str).apply(len).value_counts().sort_index()
df_tamanho_CO_ESCOLA = pd.DataFrame({'Tamanho da String CO_ESCOLA': tamanho_CO_ESCOLA.index, 'Quantidade': tamanho_CO_ESCOLA.values, "Percentual": tamanho_CO_ESCOLA / len(df_microdados_enem) })
display(formatar_dataframe(df_tamanho_CO_ESCOLA, 'Tabela 5: Tamanho da variável CO_ESCOLA', ["Tamanho da String CO_ESCOLA"], ["Quantidade"]).format({'Percentual': '{:,.2%}'}))

,Tamanho da String CO_ESCOLA,Quantidade,Percentual
CO_ESCOLA,,,
4,4,6746085,78.19%
8,8,1881282,21.81%


In [7]:
# --- Filtros combinados ---

CONCLUIRAO_EM_APOS_2016 = 3
NAO_CONCLUIRAM_EM = 4
CONCLUIRAM_ANTES_DE_2007 = 10
EXTERIOR = 4
EDUCACAO_ESPECIAL = 2
EJA = 3
TREINEIRO = 1

condicoes_filtro = (
    df_microdados_enem['TP_SEXO'].notna() &
    (df_microdados_enem['TP_ST_CONCLUSAO'] != CONCLUIRAO_EM_APOS_2016) &
    (df_microdados_enem['TP_ST_CONCLUSAO'] != NAO_CONCLUIRAM_EM) &
    (df_microdados_enem['TP_ANO_CONCLUIU'] != CONCLUIRAM_ANTES_DE_2007) &
    (df_microdados_enem['TP_ESCOLA'] != EXTERIOR) &
    (df_microdados_enem['TP_ENSINO'] != EDUCACAO_ESPECIAL) &
    (df_microdados_enem['TP_ENSINO'] != EJA) &
    (df_microdados_enem['IN_TREINEIRO'] != TREINEIRO) &
    df_microdados_enem['CO_ESCOLA'].notna() &
    df_microdados_enem['NU_NOTA_CN'].notna() &
    df_microdados_enem['NU_NOTA_CH'].notna() &
    df_microdados_enem['NU_NOTA_LC'].notna() &
    df_microdados_enem['NU_NOTA_MT'].notna() &
    df_microdados_enem['NU_NOTA_REDACAO'].notna()
)

df_filtrado = df_microdados_enem[condicoes_filtro].copy()

In [8]:
# --- Tabela 7: Registros após filtros ---

total_original = len(df_microdados_enem)
total_apos_filtro_1 = len(df_microdados_enem[df_microdados_enem['TP_SEXO'].notna()])
total_apos_filtro_2 = len(df_microdados_enem[(df_microdados_enem['TP_ST_CONCLUSAO'] != CONCLUIRAO_EM_APOS_2016) & (df_microdados_enem['TP_ST_CONCLUSAO'] != NAO_CONCLUIRAM_EM)])
total_apos_filtro_3 = len(df_microdados_enem[df_microdados_enem['TP_ANO_CONCLUIU'] != CONCLUIRAM_ANTES_DE_2007])
total_apos_filtro_4 = len(df_microdados_enem[df_microdados_enem['TP_ESCOLA'] != EXTERIOR])
total_apos_filtro_5 = len(df_microdados_enem[df_microdados_enem['TP_ENSINO'] != EDUCACAO_ESPECIAL])
total_apos_filtro_6 = len(df_microdados_enem[df_microdados_enem['IN_TREINEIRO'] != TREINEIRO])
total_apos_filtro_7 = len(df_microdados_enem[df_microdados_enem['CO_ESCOLA'].notna()])
total_apos_filtro_8 = len(df_microdados_enem[
    df_microdados_enem['NU_NOTA_CN'].notna() &
    df_microdados_enem['NU_NOTA_CH'].notna() &
    df_microdados_enem['NU_NOTA_LC'].notna() &
    df_microdados_enem['NU_NOTA_MT'].notna() &
    df_microdados_enem['NU_NOTA_REDACAO'].notna()
])

filtros_quantidade = [total_original, total_apos_filtro_1, total_apos_filtro_2, total_apos_filtro_3, total_apos_filtro_4, total_apos_filtro_5, total_apos_filtro_6, total_apos_filtro_7, total_apos_filtro_8, len(df_filtrado)]
filtros_rotulos = ["Base original", "Com informação de gênero", "Já concluíram ou concluirão EM em 2016", "Concluíram EM após 2007", "Concluíram EM no Brasil", "Concluíram EM no ensino regular", "Não são treineiros", "Com informação presente da escola de EM", "Com informação presente de notas", "Após todos os filtros"]

df_filtros = pd.DataFrame({"Quantidade": filtros_quantidade, "Percentual": filtros_quantidade / total_original})
df_filtros.index = filtros_rotulos

df_filtros_formatado = df_filtros.style.format({'Percentual': "{:,.2%}"})
df_filtros_formatado.set_caption("Tabela 7: Tabulação de observações durante aplicação de filtros")
display(df_filtros_formatado)

MemoryError: Unable to allocate 8.23 MiB for an array with shape (8627367,) and data type bool

In [ ]:
# --- Concatenação de notas otimizada ---

df_filtrado['NOTAS_CONCATENADAS'] = df_filtrado[['NU_NOTA_LC', 'NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']].astype(str).apply(' '.join, axis=1)

In [ ]:
# --- Tabela 8: Notas Concatenadas ---

notas_concatenadas_vc = df_filtrado['NOTAS_CONCATENADAS'].value_counts(dropna=True, sort=True)
df_notas_concatenadas_vc = pd.DataFrame(notas_concatenadas_vc)
df_notas_concatenadas = df_notas_concatenadas_vc.reset_index()
df_notas_concatenadas.columns = ['Notas', 'Quantidade']
df_notas_concatenadas_formatado = df_notas_concatenadas.style.hide()
df_notas_concatenadas_formatado.set_caption("Tabela 8: Tabulação de observações de unicidade de notas")
display(df_notas_concatenadas_formatado)

In [ ]:
# --- Histogramas ---

criar_histograma(df_filtrado, 'NU_NOTA_CN', 'Histograma das Notas de Ciências da Natureza', [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], '#b34242', '#7a1010')
criar_histograma(df_filtrado, 'NU_NOTA_CH', 'Histograma das Notas de Ciências Humanas', [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], '#b36842', '#7a5010')
criar_histograma(df_filtrado, 'NU_NOTA_LC', 'Histograma das Notas de Linguagens e Códigos', [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], '#b34242', '#7a1010')
criar_histograma(df_filtrado, 'NU_NOTA_MT', 'Histograma das Notas de Matemática', [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], '#7eb342', '#477a10')
criar_histograma(df_filtrado, 'NU_NOTA_REDACAO', 'Histograma das Notas de Redação', [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], '#427bb3', '#10537a')

In [ ]:
# --- Tabela 9: UF de Residência ---

display(criar_tabela_frequencia(df_filtrado, 'SG_UF_RESIDENCIA', 'Tabela 9: Distribuição da UF de residência dos alunos'))

uf_percentual_grafico = df_filtrado['SG_UF_RESIDENCIA'].value_counts(normalize=True).apply(lambda x: f'{x:.2%}')
criar_grafico_barras(df_filtrado, "Gráfico 6: Distribuição da UF de residência dos participantes", y="SG_UF_RESIDENCIA", x='COUNT', labels=uf_percentual_grafico)

In [ ]:
# --- Sexo ---

display(criar_tabela_frequencia(df_filtrado, "TP_SEXO", "Tabela 10: Distribuição por sexo"))

In [ ]:
# --- Escolaridade dos pais ---

display(criar_tabela_frequencia(df_filtrado, "Q001", "Tabela 11: Escolaridade do Pai", q046_map))
display(criar_tabela_frequencia(df_filtrado, "Q002", "Tabela 12: Escolaridade da Mãe", q046_map))

In [ ]:
# --- Raça ---

display(criar_tabela_frequencia(df_filtrado, "TP_COR_RACA", "Tabela 13: Distribuição por raça", tp_cor_raca_dict))

In [ ]:
# --- Idade ---

idade = df_filtrado[["NU_IDADE", "TP_SEXO"]].copy()
idade = idade.dropna()

idade_masculino = idade['NU_IDADE'][idade['TP_SEXO'] == "M"]
idade_feminino = idade['NU_IDADE'][idade['TP_SEXO'] == "F"]

idade_masc_counts = idade_masculino.value_counts().sort_index()
idade_fem_counts = idade_feminino.value_counts().sort_index()

display(criar_tabela_frequencia(pd.DataFrame({'NU_IDADE': idade_masculino}), 'NU_IDADE', "Tabela 14: Idade - Masculino"))
display(criar_tabela_frequencia(pd.DataFrame({'NU_IDADE': idade_feminino}), 'NU_IDADE', "Tabela 15: Idade - Feminino"))

criar_histograma(pd.DataFrame({'NU_IDADE': idade_masculino}), 'NU_IDADE', "Histograma Idade - Masculino", cor_face='#2271b3', cor_edge='#0a334e')
criar_histograma(pd.DataFrame({'NU_IDADE': idade_feminino}), 'NU_IDADE', "Histograma Idade - Feminino", cor_face='#b3224f', cor_edge='#75092b')

In [ ]:
# --- Dependência Administrativa ---

display(criar_tabela_frequencia(df_filtrado, "TP_DEPENDENCIA_ADM_ESC", "Tabela 16: Dependência Administrativa", tp_escola_dict))

In [ ]:
# --- Análise de notas por sexo ---

colunas_notas = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']

for coluna in colunas_notas:
    nota_sexo = df_filtrado[[coluna, "TP_SEXO"]].copy()
    nota_sexo = nota_sexo.dropna()

    nota_masculino = nota_sexo[coluna][nota_sexo['TP_SEXO'] == "M"]
    nota_feminino = nota_sexo[coluna][nota_sexo['TP_SEXO'] == "F"]

    display(criar_tabela_frequencia(pd.DataFrame({coluna: nota_masculino}), coluna, f"Tabela: {coluna} - Masculino"))
    display(criar_tabela_frequencia(pd.DataFrame({coluna: nota_feminino}), coluna, f"Tabela: {coluna} - Feminino"))

    criar_histograma(pd.DataFrame({coluna: nota_masculino}), coluna, f"Histograma: {coluna} - Masculino", cor_face='#2271b3', cor_edge='#0a334e')
    criar_histograma(pd.DataFrame({coluna: nota_feminino}), coluna, f"Histograma: {coluna} - Feminino", cor_face='#b3224f', cor_edge='#75092b')